# Data Analysis and Visualization in Python
## Combining DataFrames with pandas
Questions
* Can I work with data from multiple sources?
* How can I combine data from different data sets?

Objectives
* Combine data from multiple files into a single DataFrame using merge and concat.
* Combine two DataFrames using a unique ID found in both DataFrames.
* Employ `to_csv` to export a DataFrame in CSV format.
* Join DataFrames using common fields (join keys).

## Loading our data

In [ ]:
import pandas as pd

In [ ]:
surveys_df = pd.read_csv("../data/surveys.csv")
surveys_df.head()

## Concatenating DataFrames

In [ ]:
# Read in first 10 lines of surveys table
survey_sub = surveys_df.head(10)
survey_sub

In [ ]:
# Grab the last 10 rows
survey_sub_last10 = surveys_df.tail(10)
survey_sub_last10

In [ ]:
# Reset the index values to the second dataframe appends properly
# drop=True option avoids adding new index column with old index values
survey_sub_last10 = survey_sub_last10.reset_index(drop=True)
survey_sub_last10

In [ ]:
# Stack the DataFrames on top of each other
vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis=0)
vertical_stack # You may want to reset the index again

In [ ]:
# Place the DataFrames side by side
horizontal_stack = pd.concat([survey_sub, survey_sub_last10], axis=1)
horizontal_stack

### Writing Out Data to CSV

In [ ]:
# Write DataFrame to CSV
vertical_stack.to_csv('out.csv', index=False)

In [ ]:
# For kicks read our output back into python and make sure all looks good
new_output = pd.read_csv('out.csv')
new_output

## Exercise - Concatenating DataFrames
In `surveys_df`, select rows where the year is 2001. Do the same for year 2002. Concatenate both dataframes. Create a single bar-plot that shows the average weight by sex for each year. Export your results as a CSV and make sure it reads back into python properly.

In [ ]:
# Get data for each year
survey2001 = surveys_df[surveys_df['year'] == 2001]
survey2002 = surveys_df[surveys_df['year'] == 2002]
# Concatenate vertically
survey_all = pd.concat([survey2001, survey2002], axis=0)

In [ ]:
# Get the average weight for each year, grouped by sex:
weight_year = survey_all.groupby(['year', 'sex'])["weight"].mean().unstack()
weight_year

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot:
weight_year.plot(kind="bar")
plt.tight_layout()  # tip(!)

In [ ]:
# Writing to file:
weight_year.to_csv("weight_for_year.csv")

# Reading it back in:
pd.read_csv("weight_for_year.csv", index_col=0)

## Joining DataFrames
### Joining Two DataFrames

In [ ]:
# With the first 10 lines of surveys table
survey_sub

In [ ]:
# Import a small subset of the species data designed for this part of the lesson
species_sub = pd.read_csv('../data/speciesSubset.csv')
species_sub

### Identifying join keys

In [ ]:
survey_sub.columns

In [ ]:
species_sub.columns

### Inner joins

![Inner join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/inner-join.png)

In [ ]:
merged_inner = pd.merge(left=survey_sub, right=species_sub,
                        left_on='species_id', right_on='species_id')
# What's the size of the output data?
merged_inner.shape

In [ ]:
merged_inner

### Left joins

![Left join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/left-join.png)

In [ ]:
merged_left = pd.merge(left=survey_sub, right=species_sub, how='left',
                       left_on='species_id', right_on='species_id')
# What's the size of the output data?
merged_left.shape

In [ ]:
merged_left

In [ ]:
merged_left[ pd.isnull(merged_left['genus']) ]

### Other join types
* `how='right'` : all rows from the right DataFrame are kept
* `how='outer'` : all pairwise combinations of rows from both DataFrames

## Exercise - Joining all data
`1`. Create a new DataFrame by joining the contents of the `surveys.csv` and `species.csv` tables.

In [ ]:
species_df = pd.read_csv("../data/species.csv")
merged_left = pd.merge(left=surveys_df, right=species_df, how='left', on="species_id")

`2`. Calculate and plot the distribution of taxa by site (i.e. the number of different `taxa` per site "`plot_id`")

In [ ]:
merged_left.groupby(["plot_id"])["taxa"].nunique().plot(kind='bar')

`3`. Calculate and plot the distribution of surveys (i.e. the number of record IDs) by taxa by site

In [ ]:
taxa_site = merged_left.groupby(["plot_id", "taxa"])["record_id"].count().unstack()
taxa_site

In [ ]:
taxa_site.plot(kind='bar', stacked=True)
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.05))

`4`. Calculate and plot the distribution of taxa by sex by site

In [ ]:
# Data cleanup
merged_left.loc[merged_left["sex"].isnull(), "sex"] = 'M|F'
merged_left.loc[~merged_left["sex"].isin(['F', 'M', 'M|F']), "sex"] = 'M|F'

In [ ]:
ntaxa_sex_site = merged_left.groupby(["plot_id", "sex"])["taxa"].nunique().reset_index(level=1)
ntaxa_sex_site

In [ ]:
# Use pivot_table() instead of unstack()
ntaxa_sex_site = ntaxa_sex_site.pivot_table(values="taxa", columns="sex",
                                            index=ntaxa_sex_site.index)
ntaxa_sex_site

In [ ]:
ntaxa_sex_site.plot(kind="bar", stacked=True, legend=False)
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.08),
           fontsize='small', frameon=False)